In [3]:
import pandas as pd
import numpy as np
import FinanceDataReader as fdr
import psycopg2

import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from collections import Counter
from bs4 import BeautifulSoup
import requests
import time

In [4]:
# kospi 전체 종목 목록 불러오기
df_kospi_master = fdr.StockListing('KOSPI-DESC')

# 우선주 걸러내기
df_kospi_master = df_kospi_master[df_kospi_master["Sector"].notnull()]
df_kospi_master.reset_index(inplace= True)
df_kospi_master = df_kospi_master.drop(df_kospi_master.columns[0], axis=1)
df_kospi_master

,Code,Name,Market,Sector,Industry,ListingDate,SettleMonth,Representative,HomePage,Region
0,095570,AJ네트웍스,KOSPI,산업용 기계 및 장비 임대업,"렌탈(파렛트, OA장비, 건설장비)",2015-08-21,12월,손삼달,http://www.ajnet.co.kr,서울특별시
1,006840,AK홀딩스,KOSPI,기타 금융업,지주사업,1999-08-11,12월,"채형석, 백차현(각자 대표이사)",http://www.aekyunggroup.co.kr,서울특별시
2,027410,BGF,KOSPI,기타 금융업,지주회사,2014-05-19,12월,홍정국,http://www.bgf.co.kr,서울특별시
3,282330,BGF리테일,KOSPI,종합 소매업,체인화 편의점,2017-12-08,12월,민승배,http://www.bgfretail.com,서울특별시
4,138930,BNK금융지주,KOSPI,기타 금융업,금융지주회사,2011-03-30,12월,빈대인,http://www.bnkfg.com,부산광역시
...,...,...,...,...,...,...,...,...,...,...
837,005870,휴니드,KOSPI,통신 및 방송 장비 제조업,"무선통신기기,군납용무전기,위성방송수신기 제조",1991-09-03,12월,신종석,http://www.huneed.com,인천광역시
838,079980,휴비스,KOSPI,화학섬유 제조업,"합성섬유(폴리에스테르원사,원면),재생섬유,폴리에스텔 원사,원면,고상칩 제조,도소매",2012-02-23,12월,김석현,http://www.huvis.com,서울특별시
839,005010,휴스틸,KOSPI,1차 철강 제조업,"강관(배관용,구조용,유정용) 제조,도매",1973-06-29,12월,박훈,http://www.husteel.com,서울특별시
840,000540,흥국화재,KOSPI,보험업,손해보험,1974-12-05,12월,송윤상,http://www.insurance.co.kr,서울특별시


In [5]:
# kosdaq 전체 종목 목록 불러오기
df_kosdaq_master = fdr.StockListing('KOSDAQ-DESC')

# 스팩주 제외하기
df_kosdaq_master = df_kosdaq_master[~df_kosdaq_master["Name"].str.contains('스팩')]
df_kosdaq_master.reset_index(inplace = True)
df_kosdaq_master = df_kosdaq_master.drop(df_kosdaq_master.columns[0], axis=1)
df_kosdaq_master

,Code,Name,Market,Sector,Industry,ListingDate,SettleMonth,Representative,HomePage,Region
0,060310,3S,KOSDAQ,전자부품 제조업,반도체 웨이퍼 캐리어,2002-04-23,03월,김세완,http://www.3sref.com,서울특별시
1,054620,APS,KOSDAQ,기타 금융업,인터넷 트래픽 솔루션,2001-12-04,12월,정기로,http://www.apsholdings.co.kr,경기도
2,265520,AP시스템,KOSDAQ,특수 목적용 기계 제조업,디스플레이 제조 장비,2017-04-07,12월,김영주,http://www.apsystems.co.kr,경기도
3,211270,AP위성,KOSDAQ,"항공기,우주선 및 부품 제조업",위성통신 단말기,2016-03-04,12월,류장수,http://www.apsi.co.kr,서울특별시
4,109960,AP헬스케어,KOSDAQ,기타 전문 도매업,"모바일게임 퍼블리싱, 바이오의약품 연구개발업 및 판매, 생산설비 및 연구장비의 대여...",2009-11-13,12월,김정출,http://www.aprogen-hng.com,경기도
...,...,...,...,...,...,...,...,...,...,...
1595,024060,흥구석유,KOSDAQ,기타 전문 도매업,휘발유등,1994-12-07,12월,김상우,NaN,대구광역시
1596,010240,흥국,KOSDAQ,특수 목적용 기계 제조업,굴삭기 Roller 및 형단조품,2009-05-12,12월,류명준,http://www.heungkuk.co.kr,충청남도
1597,189980,흥국에프엔비,KOSDAQ,비알코올음료 및 얼음 제조업,"과일음료(에이드베이스, 스무디, 착즙쥬스 등)",2015-08-07,12월,"박철범, 오길영",http://www.hyungkuk.com,서울특별시
1598,037440,희림,KOSDAQ,"건축기술, 엔지니어링 및 관련 기술 서비스업",설계 및 감리용역,2000-02-03,12월,"정영균, 이목운, 허철호 (각자대표)",http://www.heerim.com,서울특별시


In [6]:
# 일별 주가 받아오기
kospi_day_price_list = []
kosdaq_day_price_list = []

for i in range(len(df_kospi_master)):
    stock_name = df_kospi_master["Name"][i]
    stock_code = df_kospi_master["Code"][i]
    df_stock_price = fdr.DataReader('KRX:'+stock_code, '2015-06-15') # 데이터소스:종목코드, 시작일, 종료일
    df_stock_price.reset_index(inplace = True)
    df_stock_price["Name"] = stock_name
    df_stock_price["Code"] = stock_code
    kospi_day_price_list.append(df_stock_price)
  
for i in range(len(df_kosdaq_master)):
    stock_name = df_kosdaq_master["Name"][i]
    stock_code = df_kosdaq_master["Code"][i]
    df_stock_price = fdr.DataReader('KRX:'+stock_code, '2015-06-15')
    df_stock_price.reset_index(inplace = True)
    df_stock_price["Name"] = stock_name
    df_stock_price["Code"] = stock_code
    kosdaq_day_price_list.append(df_stock_price)


In [27]:
kospi_day_price_list[0].to_excel("data/day_price/냥.xlsx", index=False)

In [28]:
# 받아온 일별 주가 엑셀파일로 저장하기(kospi)
save_path = "data/day_price/kospi/"
for i in range(len(kospi_day_price_list)):
    df_stock_price = kospi_day_price_list[i]
    stock_name = df_stock_price["Name"][0]
    df_stock_price.to_excel(save_path + stock_name + ".xlsx", index= False)
    if (i+1) % 100 == 0:
        print("{}/{}".format(i+1, len(kospi_day_price_list)))

1/842
101/842
201/842
301/842
401/842
501/842
601/842
701/842
801/842


In [29]:
# 받아온 일별 주가 엑셀파일로 저장하기(kosdaq)
save_path = "data/day_price/kosdaq/"
for i in range(len(kosdaq_day_price_list)):
    df_stock_price = kosdaq_day_price_list[i]
    stock_name = df_stock_price["Name"][0]
    df_stock_price.to_excel(save_path + stock_name + ".xlsx", index= False)
    if (i+1) % 100 == 0:
        print("{}/{}".format(i+1, len(kosdaq_day_price_list)))

100/1600
200/1600
300/1600
400/1600
500/1600
600/1600
700/1600
800/1600
900/1600
1000/1600
1100/1600
1200/1600
1300/1600
1400/1600
1500/1600
1600/1600


In [ ]:
# 뉴스 제목 추출(kospi)

save_path = "data/news_and_keyword/kospi/"
for i in range(len(kospi_day_price_list)):
    stock_name = kospi_day_price_list[i]["Name"][0]
    df_stock_price = kospi_day_price_list[i]

    date_list = []
    title_list = []
    keyword_list = []

    for j in range(df_stock_price):
        date = df_stock_price["Date"][j]
        # 일별 뉴스 검색하기
        add_element = "[특징주] "
        link = "https://search.naver.com/search.naver?where=news&query="+ add_element + stock_name + "&sm=tab_opt&sort=0&photo=0&field=0&pd=3&ds=" + date + "&de=" + date +"&docid=&related=0&mynews=0&office_type=0&office_section_code=0&news_office_checked=&nso=so%3Ar%2Cp%3Afrom20240628to20240628&is_sug_officeid=0&office_category=0&service_area=0"
        response = requests.get(link, headers={"User-Agent" : "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.0.0 Safari/537.36"})
        html = response.content
        soup = BeautifulSoup(html, "html.parser")
        page = soup.find_all("a", {"class" : "news_tit"})
      
        total_title = ""

        if len(page) > 0: # 해당 날짜에 기사가 있으면
            for i in range(len(page)):
                title = page[i].get_text()
                if stock_name in title: # 기사 제목에 종목명이 포함된거만 빼냄, 같은 날짜의 기사가 여러 개일 수 있음, 기사를 한꺼번에 모아서 키워드 추출할 준비
                    total_title += (title + " ")
            date_list.append(date)
            title_list.append(total_title)
            
            keywords = get_keyword(date, total_title, stock_name) # 기사 제목 키워드 추출

            keyword_list.append(keyword)


In [45]:
stock_name = kospi_day_price_list[0]["Name"][0]
df_stock_price = kospi_day_price_list[0]

date_list = []
title_list = []
keyword_list = []

for j in range(len(df_stock_price)):
    date = str(df_stock_price["Date"][j])[0:10].replace("-","")
    # 일별 뉴스 검색하기
    add_element = "[특징주] "
    link = "https://search.naver.com/search.naver?where=news&query="+ add_element + stock_name + "&sm=tab_opt&sort=0&photo=0&field=0&pd=3&ds=" + date + "&de=" + date +"&docid=&related=0&mynews=0&office_type=0&office_section_code=0&news_office_checked=&nso=so%3Ar%2Cp%3Afrom20240628to20240628&is_sug_officeid=0&office_category=0&service_area=0"
    response = requests.get(link, headers={"User-Agent" : "Mozilla/5.0"})
    html = response.content
    soup = BeautifulSoup(html, "html.parser")
    page = soup.find_all("a", {"class" : "news_tit"})
    
    total_title = ""

    if len(page) > 0: # 해당 날짜에 기사가 있으면
        for i in range(len(page)):
            title = page[i].get_text()
            if stock_name in title: # 기사 제목에 종목명이 포함된거만 빼냄, 같은 날짜의 기사가 여러 개일 수 있음, 기사를 한꺼번에 모아서 키워드 추출할 준비
                total_title += (title + " ")
        date_list.append(date)
        title_list.append(total_title)
        
        keywords = get_keyword(date, total_title, stock_name) # 기사 제목 키워드 추출

        keyword_list.append(keywords)
    
    print("{} / {} / {} / {}".format(j, len(df_stock_price), date, total_title))
    time.sleep(1)

0 / 2192 / 20150821 / [특징주]AJ네트웍스, 상장 첫날 주가 약세 [특징주]AJ네트웍스, 상장 첫날 약세…시초가 공모가 하회 [특징주] AJ네트웍스, 상장 첫날 공모가 하회 후 ↓ [특징주]AJ네트웍스, 코스피상장 첫날 약세 출발…공모가 하회 [fn특징주]AJ네트웍스, 코스피 상장 첫날 공모가 하회 출발 (특징주)AJ네트웍스, 상장 첫날 약세 [특징주]AJ네트웍스, 상장 첫날 공모가 밑돌아 [특징주] AJ네트웍스, 코스피 상장 첫날 공모가 하회 
1 / 2192 / 20150824 / [특징주]AJ네트웍스, 상장 이틀째 하락반전...공모가 하회 [특징주]AJ네트웍스, 상장 이튿날 급락 [특징주]AJ네트웍스, 상장 2거래일 만에 급락 
2 / 2192 / 20150825 / 
3 / 2192 / 20150826 / [특징주]AJ네트웍스, 상장 나흘째 상승반전..외인 '사자' 
4 / 2192 / 20150827 / 
5 / 2192 / 20150828 / 
6 / 2192 / 20150831 / 
7 / 2192 / 20150901 / [특징주]AJ네트웍스, 상승 전환… 렌탈 시장 지배력 ‘매력’ 
8 / 2192 / 20150902 / 
9 / 2192 / 20150903 / 
10 / 2192 / 20150904 / 
11 / 2192 / 20150907 / 
12 / 2192 / 20150908 / 
13 / 2192 / 20150909 / 
14 / 2192 / 20150910 / 
15 / 2192 / 20150911 / 
16 / 2192 / 20150914 / 
17 / 2192 / 20150915 / 
18 / 2192 / 20150916 / 
19 / 2192 / 20150917 / 
20 / 2192 / 20150918 / 
21 / 2192 / 20150921 / 
22 / 2192 / 20150922 / 
23 / 2192 / 20150923 / 
24 / 2192 / 20150924 / 
25 / 2192 / 201509

KeyboardInterrupt: 

In [38]:
df_stock_price[1]

,Date,Open,High,Low,Close,Volume,Change,UpDown,Comp,Amount,MarCap,Shares,Name,Code
0,2015-08-21,6180,7300,5600,7100,1994022,0.1489,1,920,64841739950,332438294500,9364459,AJ네트웍스,095570
1,2015-08-24,6700,6940,6240,6480,392831,-0.0873,2,-620,12926256800,303408471600,9364459,AJ네트웍스,095570
2,2015-08-25,6610,6730,6190,6220,242730,-0.0401,2,-260,7808534250,291234674900,9364459,AJ네트웍스,095570
3,2015-08-26,6260,7760,6260,7090,703721,0.1399,1,870,25144972800,331970071550,9364459,AJ네트웍스,095570
4,2015-08-27,7240,8060,6960,7570,499913,0.0677,1,480,18764860300,354444773150,9364459,AJ네트웍스,095570
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2187,2024-07-15,4550,4650,4500,4550,80868,0.0000,3,0,368643620,205900053450,45252759,AJ네트웍스,095570
2188,2024-07-16,4550,4590,4465,4535,96306,-0.0033,2,-15,432125715,205221262065,45252759,AJ네트웍스,095570
2189,2024-07-17,4530,4535,4380,4460,128190,-0.0165,2,-75,569540225,201827305140,45252759,AJ네트웍스,095570
2190,2024-07-18,4415,4430,4355,4365,198888,-0.0213,2,-95,870754925,197528293035,45252759,AJ네트웍스,095570


In [36]:
str(df_stock_price["Date"][0])[0:10].replace("-","")

'20150821'

In [30]:
def get_keyword(date, title, stock_name): # 종목명과 기사제목을 받아 키워드를 추출함
    # 직접 정의한 한국어 불용어 목록
    stop_words = set([
        '이', '그', '저', '것', '수', '등', '들', '및', '의', '를', '에', '과', '와', '한', 
        '하다', '되다', '을', '를', '은', '는', '이', '가', '로', '으로', '에게', '께', '에서',
        '이다', '하다', '것', '있다', '되다', '수', '나', '우리', '너', '있다', '이다', '것', '특징주'
    ])
    stop_words.add(stock_name) # 불용어 목록에 종목명 추가(기사에 포함된 종목명이 키워드에 걸리지 않기 위해)

    words = word_tokenize(title)

    filtered_words = [word for word in words if word.isalnum() and word not in stop_words] # 불용어 및 특수문자 제거
    word_freq = Counter(filtered_words) # 단어 빈도 계산
    keywords = [word for word, freq in word_freq.most_common(5)] # 키워드 추출 (빈도순 상위 5개 단어)
    return keywords